This notebook is designed to reproduce several findings from Ted Underwood and Jordan Sellers's article "How Quickly Do Literary Standards Change?" (draft (2015), forthcoming in <i>Modern Language Quarterly</i>). See especially Fig 1 and reported classifier accuracy (p 8).

Underwood and Sellers have made their corpus of poems and their code available here: https://github.com/tedunderwood/paceofchange

## Brief Intro to Logistic Regression
<li>Intuiting the Linear Regression Model</li>
<li>Intuiting the Logistic Regression Model</li>
<li>Logistic Classification</li>

## Cross-Validation
<li>Pre-Processing</li>
<li>Feature Selection, Training, Predictions</li>
<li>Efficiency</li>
<li>Evaluation</li>

## Classification
<li>The Canon</li>

# Brief Intro to Logistic Regression

In [ ]:
%pylab inline
from datascience import *
matplotlib.style.use('ggplot')

In [ ]:
# Dummy Data Set

demo_tb = Table()

demo_tb['Study Hours'] = [2.0, 6.9, 1.6, 7.8, 3.1, 5.8, 3.4, 8.5, 6.7, 1.6, 8.6, 3.4, 9.4, 5.6, 9.6, 3.2, 3.5, 5.9, 9.7, 6.5]
demo_tb['Grade'] = [67.0, 83.6, 35.4, 79.2, 42.4, 98.2, 67.6, 84.0, 93.8, 64.4, 100.0, 61.6, 100.0, 98.4, 98.4, 41.8, 72.0, 48.6, 90.8, 100.0]
demo_tb['Pass'] = [0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1]

demo_tb

## Intuiting the Linear Regression Model

In [ ]:
# Input  >> Continuous Variable
# Output >> Continuous Variable

# Tries to find underlying linear relationship between inputs and outputs

demo_tb.scatter('Study Hours','Grade')

In [ ]:
demo_tb.scatter('Study Hours','Grade', fit_line=True)

# Intuiting the Logistic Regression Model

In [ ]:
# Input  >> Continuous Variable
# Output >> Categorical Variable

# Tries to find probability of an output given an input

demo_tb.scatter('Study Hours','Pass')

In [ ]:
# Define the Logistic Function

def logistic(p):
    return 1/(1+exp(-p))

# Assign generic Logistic Regression Coefficients

B0, B1 = 0,1

# Clunky Plotting for Continuous Functions

points = int(1e4)
xmin, xmax = -10,10
xlist = [float(x)/points for x in range(xmin*points, xmax*points)]
ylist = [logistic(B0 + B1*x) for x in xlist]

axis([-10, 10, -0.1,1.1])
plot(xlist,ylist)

In [ ]:
# Calculate Logistic Regression Coefficients

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(demo_tb['Study Hours'].reshape(-1,1),demo_tb['Pass'].reshape(20,))
B0, B1 = lr.intercept_[0], lr.coef_[0]

# Clunky Plotting for Continuous Functions

points = int(1e4)
xmin, xmax = 0,10
xlist = [float(x)/points for x in range(xmin*points, xmax*points)]
ylist = [logistic(B0 + B1*x) for x in xlist]
plot(xlist,ylist)

# Add our "Observed" Data Points

scatter(demo_tb['Study Hours'],demo_tb['Pass'])

## Logistic Classification

In [ ]:
# Create training and test sets

train_hours = demo_tb.column('Study Hours')[:-2]
train_targets = demo_tb.column('Pass')[:-2]

test_hours = demo_tb.column('Study Hours')[-2:]
test_targets = demo_tb.column('Pass')[-2:]

In [ ]:
# Check the values we are using to validate the model

print(test_hours, test_targets)

In [ ]:
# Calculate Logistic Regression Coefficients

lr.fit(train_hours.reshape(-1,1),train_targets.reshape(len(train_targets),))
B0, B1 = lr.intercept_[0], lr.coef_[0]

# Determing the probability each student would pass, based on our model
fitted = [logistic(B1*th + B0) for th in test_hours]

# See whether that probability is greater than 50% (i.e. the most likely outcome)
prediction = [pred>.5 for pred in fitted]

In [ ]:
# Test each prediction against the true answer in the validation set

prediction_eval = [prediction[i]==test_targets[i] for i in range(len(prediction))]
float(sum(prediction_eval)/len(prediction_eval))

In [ ]:
# EX. Put the values from the "Study Hours" column into standard units.

# EX. Reproduce the Logistic Regression graph using "observed" values in
#     standard units.

#  Q. In the previous exercise, what does the new graph represent?
#     What is happening at x = 0 ?

# Pre-Processing

In [ ]:
import pandas as pd
import numpy as np
corpus_path = 'poems/'

In [ ]:
#metadata_tb = pd.read_csv('poemeta.csv')
metadata_tb = Table.read_table('poemeta.csv', keep_default_na=False)

In [ ]:
metadata_tb

In [ ]:
# Select the records that we will eventually try to classify

reception_mask = (metadata_tb['recept']=='reviewed') + (metadata_tb['recept']=='random')
clf_tb = metadata_tb.where(reception_mask)

clf_tb

In [ ]:
# Import Term Frequencies by Text


# NOTE: This script is specifically tailored to the format of 
# textual data made available from Hathi Trust. This consists of a 
# series ofspreadsheets, each containing book-level term frequencies.

# Each spreadsheet will become a row in our Document-Term Matrix.


# Create list that will contain a series of dictionaries
freqdict_list = []

# Iterate through texts in our spreadsheet
for _id in clf_tb['docid']:

    # Each text will have its own dictionary
    # Keys are terms and values are frequencies
    termfreq_dict = {}
    
    # Open the given text's spreadsheet
    with open(corpus_path+_id+'.poe.tsv', encoding='utf-8') as file_in:
        filelines = file_in.readlines()
        
        # Each line in the spreadsheet contains a unique term and its frequency
        for line in filelines:
            termfreq = line.split('\t')
            
            # 'If' conditions throw out junk lines in the spreadsheet
            if len(termfreq) > 2 or len(termfreq) > 2:
                continue
            term, freq = termfreq[0], int(termfreq[1])
            if len(term)>0 and term[0].isalpha():
                
                # Create new entry in text's dictionary for the term
                termfreq_dict[term] = freq
                
    freqdict_list.append(termfreq_dict)

In [ ]:
# Convert our list of term-frequency dictionaries into a document-term matrix

from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer()
dtm = dv.fit_transform(freqdict_list)
term_list = dv.feature_names_

In [ ]:
dtm

In [ ]:
# Put our Document-Term-Matrix into human-readable format

dtm_tb = Table(term_list).with_rows(dtm.toarray())

# Feature Selection, Training, Prediction

In [ ]:
# Pandas DataFrame -- not Table -- since offers additional methods

import pandas as pd
dtm_df = pd.DataFrame(dtm.toarray(), columns = term_list)

In [ ]:
# Use the unique IDs from our metadata to keep track of each text

dtm_df.set_index(clf_tb['docid'], inplace=True)
dtm_df

## Inputs and Outputs

In [ ]:
# Underwood and Sellers create a unique model for each author in the corpus.
# They set aside a given author from the training set and then use the model to
# predict whether she was likely to be reviewed or not.

# Create a list of authors and an "empty" array in which to record probabilities

authors = list(set(clf_tb['author']))
probabilities = np.zeros([len(clf_tb['docid'])])

In [ ]:
## Logistic Regressor

# Underwood and Sellers use a regularization constant ('C') to prevent overfitting,
# since this is a major concern when observing thousands of variables.

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C = 0.00007)

## Feature Selection

In [ ]:
# Remember: Each author's model potentially consists of a unique set of 3200 words.


# Set aside each author's texts from training set

def set_author_aside(author, tb, df):
    train_ids = tb.where(tb['author']!=author).column('docid')
    test_ids = tb.where(tb['author']==author).column('docid')
    
    train_df_ = df.loc[train_ids]
    test_df_ = df.loc[test_ids]
    
    train_targets_ = tb.where(tb['author']!=author)['recept']=='reviewed'
    
    return train_df_, test_df_, train_targets_


# Retrieve the most common words (by document frequency) for a given model

def top_vocab_by_docfreq(df, num_words):
    docfreq_df = df > 0
    wordcolumn_sums = sum(docfreq_df)
    words_by_freq = wordcolumn_sums.sort_values(ascending=False)
    top_words = words_by_freq[:num_words]
    top_words_list = top_words.index.tolist()
    
    return top_words_list


# Normalize the model's term frequencies and put them into standard units

def normalize_model(train_df_, test_df_, vocabulary):
    # Select columns for only the most common words
    train_df_ = train_df_[vocabulary]
    test_df_ = test_df_[vocabulary]
    
    # Normalize each value by the sum of all values in its row
    train_df_ = train_df_.apply(lambda x: x/sum(x), axis=1)
    test_df_ = test_df_.apply(lambda x: x/sum(x), axis=1)
    
    # Get mean and stdev for each column
    train_mean = np.mean(train_df_)
    train_std = np.std(train_df_)

    # Transform each value to standard units for its column
    train_df_ = ( train_df_ - train_mean ) / train_std
    test_df_ = ( test_df_ - train_mean ) / train_std
    
    return train_df_, test_df_

In [ ]:
## EX. Create a new function that determines top vocabulary by each word's
##     total frequency in the training set.

## Training and Prediction

In [ ]:
# We're going to time this to make a point about computational efficiency
import time
start = time.clock()

for author in authors[:10]:
    
    # Set aside each author's texts from training set
    train_df, test_df, train_targets = set_author_aside(author, clf_tb, dtm_df)

    # Retrieve the most common words (by document frequency) for a given model
    vocab_list = top_vocab_by_docfreq(train_df, 3200)
    
    # Normalize the model's term frequencies and put them into standard units
    train_df, test_df = normalize_model(train_df, test_df, vocab_list)
    
    # Learn the Logistic Regression over our model
    clf.fit(train_df, train_targets)
    
    # Some authors have more than one text in the corpus, so we retrieve all
    for _id in test_df.index.tolist():
        
        # Make prediction whether text was reviewed
        text = test_df.loc[_id]
        probability = clf.predict_proba([text])[0][1]

        # Record predictions in same order as the metadata spreadsheet
        _index = list(clf_tb.column('docid')).index(_id)
        probabilities[_index] = probability
    
        
end = time.clock()
print(end - start)

In [ ]:
print(len(authors))

In [ ]:
##  Q. Why are we taking such pains to determine our vocabulary list
##     without using texts from the test set? Would it make a big difference?

##  Q. How can we make this process more efficient?

# Efficiency

## Vocabulary Size

In [ ]:
len(term_list)

In [ ]:
# Pre-Processed Vocabulary
# Contains only words that will be used in classification

# This list was created by simply iterating through each model
# and observing the words that appeared in it

import pickle
with open('preprocessed_vocab.pickle', 'rb') as f:
    pp_vocab = pickle.load(f)

In [ ]:
len(pp_vocab)

In [ ]:
# Pandas DataFrame -- not Table -- since offers additional methods

dtm_df = pd.DataFrame(dtm.toarray(), columns = term_list)

In [ ]:
# Select only columns for words in our pre-processed vocabulary
# This will make our computation more efficient later

# dtm_tb = dtm_tb.select(pp_vocab)
dtm_df = dtm_df[pp_vocab]

In [ ]:
# Use the unique IDs from our metadata to keep track of each text

dtm_df.set_index(clf_tb['docid'], inplace=True)
dtm_df

In [ ]:
## EX. Using code from our previous lessons, remove stopwords from our pre-processed vocabulary.

## Document Frequency

In [ ]:
# Create new DataFrame that simply lists whether a term appears in
# each document, so that we don't have to repeat this process evey iteration

term_in_doc_df = dtm_df>0

In [ ]:
term_in_doc_df

In [ ]:
# Re-write the model-building function

def set_author_aside(author, tb, dtm_df_, dfreq_df_):
    train_ids = tb.where(tb['author']!=author).column('docid')
    test_ids = tb.where(tb['author']==author).column('docid')
    
    train_df_ = dtm_df_.loc[train_ids]
    dfreq_df_ = dfreq_df_.loc[train_ids] # Include only term_in_doc values for texts in training set
    test_df_ = dtm_df_.loc[test_ids]
    
    train_targets_ = tb.where(tb['author']!=author)['recept']=='reviewed'
    
    return train_df_, test_df_, train_targets_, dfreq_df_


# Re-write our vocabulary selection function

def top_vocab_by_docfreq(df, num_words):
    # Removed the test of whether a term is in a given document (i.e. df>0)
    wordcolumn_sums = sum(df)
    words_by_freq = wordcolumn_sums.sort_values(ascending=False)
    top_words = words_by_freq[:num_words]
    top_words_list = top_words.index.tolist()
    
    return top_words_list

## Parallel Processing

In [ ]:
clf = LogisticRegression(C = 0.00007)

def master_function(author):
    # Note: Our only input is the name of the author.
    # Remember that we had iterated over the list of authors previously.
    train_df, test_df, train_targets, dfreq_df = set_author_aside(author, clf_tb, dtm_df, term_in_doc_df)
    vocab_list = top_vocab_by_docfreq(dfreq_df, 3200)
    train_df, test_df = normalize_model(train_df, test_df, vocab_list)
    clf.fit(train_df, train_targets)
    
    # Create a list of each text's probability of review AND its index in the metadata table
    index_probability_tuples = []
    for _id in test_df.index.tolist():
        text = test_df.loc[_id]
        probability = clf.predict_proba([text])[0][1]
        _index = list(clf_tb.column('docid')).index(_id)
        index_probability_tuples.append( (_index, probability) )
    return index_probability_tuples

In [ ]:
# Multiprocessing enables Python to run its script on multiple cores simultaneously
# This is best used in situations where we might otherwise use a 'FOR' loop.

import multiprocessing

# Return number of cores
multiprocessing.cpu_count()

In [ ]:
# The Pool contains one worker for each core
pool = multiprocessing.Pool()

# Efficiently applies the master_function() to our list of authors
# Returns a list where each entry is an item returned by the function
output = pool.map(master_function, authors)

In [ ]:
output[:10]

In [ ]:
# Each element in output is itself a list, the length of which is the number
# of texts by a given author. We'll flatten it for ease of use.

flat_output = [tup  for lst in output  for tup in lst]
flat_output[:10]

In [ ]:
# Use the indices returned with the output to arrange probabilities properly

probabilities = np.zeros([len(clf_tb['docid'])])
for tup in flat_output:
    probabilities[tup[0]] = tup[1]

clf_tb['P(reviewed)'] = probabilities

In [ ]:
clf_tb.select(['docid', 'firstpub','author', 'title', 'recept', 'P(reviewed)'])

## Evaluation

In [ ]:
# Visualize the probability each text was reviewed

colors = ['r' if recept=='reviewed' else 'b'  for recept in clf_tb['recept']]

clf_tb.scatter('firstpub', 'P(reviewed)', c=colors, fit_line=True)

In [ ]:
# Does the Logistic Regression Model think its likely each book was reviewed?
predictions = probabilities>0.5
predictions

In [ ]:
from sklearn.metrics import accuracy_score

# Creates array where '1' indicates a reviewed book and '0' indicates not
targets = clf_tb['recept']=='reviewed'

print(accuracy_score(predictions, targets))

In [ ]:
# Note: Often we prefer to evaluate accuracy based on the F1-score, which
# weighs the number of times we correctly predicted reviewed texts against
# the number of times we incorrectly predicted them as 'random'.

from sklearn.metrics import f1_score

print(f1_score(predictions, targets))

In [ ]:
## EX. Change the regularization parameter ('C') in our Logistic Regression function.
##     How does this change the classifier's accuracy?

## EX. Reduce the size of the vocabulary used for classification. How does accuracy change?

In [ ]:
##  Q. Are there cases when we might not want to set the classification threshold
##     to 50% likelihood? How certain are we that 51% is different from a 49% probability?

# Classification

In [ ]:
# Train model using full set of 'reviewed' and 'random' texts

# Use this to predict the probability that other prestigious texts
# (i.e. ones that we haven't trained on) might have been reviewed
# at their times of publication.

In [ ]:
# Re-run script from scratch

%pylab inline
matplotlib.style.use('ggplot')

from datascience import *
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer

corpus_path = 'poems/'

In [ ]:
# Read metadata from spreadsheet
metadata_tb = Table.read_table('poemeta.csv', keep_default_na=False)

In [ ]:
# We'll copy just our new texts into a separate table as well, for later
canon_tb = metadata_tb.where('recept','addcanon')

In [ ]:
# Read Term Frequencies from files

freqdict_list = []

# Iterate through texts in our spreadsheet
for _id in metadata_tb['docid']:

    # Each text will have its own dictionary
    # Keys are terms and values are frequencies
    termfreq_dict = {}
    
    # Open the given text's spreadsheet
    with open(corpus_path+_id+'.poe.tsv', encoding='utf-8') as file_in:
        filelines = file_in.readlines()
        
        # Each line in the spreadsheet contains a unique term and its frequency
        for line in filelines:
            termfreq = line.split('\t')
            
            # 'If' conditions throw out junk lines in the spreadsheet
            if len(termfreq) > 2 or len(termfreq) > 2:
                continue
            term, freq = termfreq[0], int(termfreq[1])
            if len(term)>0 and term[0].isalpha():
                
                # Create new entry in text's dictionary for the term
                termfreq_dict[term] = freq
                
    freqdict_list.append(termfreq_dict)

In [ ]:
# Create the Document-Term-Matrix

dv = DictVectorizer()
dtm = dv.fit_transform(freqdict_list)
term_list = dv.feature_names_

In [ ]:
# Place the DTM into a Pandas DataFrame for further manipulation

dtm_df = pd.DataFrame(dtm.toarray(), columns = term_list)
dtm_df.set_index(metadata_tb['docid'], inplace=True)

In [ ]:
# These are Feature Selection functions like the ones we originally defined,
# not their efficiency minded counterparts, since we only train once


# Set aside each canonic texts from training set

def set_canon_aside(tb, df):
    train_ids = tb.where(tb['recept']!='addcanon').column('docid')
    classify_ids = tb.where(tb['recept']=='addcanon').column('docid')
    
    train_df_ = df.loc[train_ids]
    classify_df_ = df.loc[classify_ids]
    
    train_targets_ = tb.where(tb['recept']!='addcanon')['recept']=='reviewed'
    
    return train_df_, classify_df_, train_targets_


# Retrieve the most common words (by document frequency) for a given model

def top_vocab_by_docfreq(df, num_words):
    docfreq_df = df > 0
    wordcolumn_sums = sum(docfreq_df)
    words_by_freq = wordcolumn_sums.sort_values(ascending=False)
    top_words = words_by_freq[:num_words]
    top_words_list = top_words.index.tolist()
    
    return top_words_list


# Normalize the model's term frequencies and put them into standard units

def normalize_model(train_df_, classify_df_, vocabulary):
    # Select columns for only the most common words
    train_df_ = train_df_[vocabulary]
    classify_df_ = classify_df_[vocabulary]
    
    # Normalize each value by the sum of all values in its row
    train_df_ = train_df_.apply(lambda x: x/sum(x), axis=1)
    classify_df_ = classify_df_.apply(lambda x: x/sum(x), axis=1)
    
    # Get mean and stdev for each column
    train_mean = np.mean(train_df_)
    train_std = np.std(train_df_)

    # Transform each value to standard units for its column
    train_df_ = ( train_df_ - train_mean ) / train_std
    classify_df_ = ( classify_df_ - train_mean ) / train_std
    
    return train_df_, classify_df_

In [ ]:
# Train our Logistic Regression Model

clf = LogisticRegression(C = 0.00007)

model_df, classify_df, model_targets = set_canon_aside(metadata_tb, dtm_df)
vocab_list = top_vocab_by_docfreq(model_df, 3200)
model_df, classify_df = normalize_model(model_df, classify_df, vocab_list)
clf.fit(model_df, model_targets)

In [ ]:
# Predict whether our new prestigious texts might have been reviewed

probabilities = numpy.zeros([len(canon_tb.column('docid'))])
for _id in classify_df.index.tolist():
    text = classify_df.loc[_id]
    probability = clf.predict_proba([text])[0][1]
    
    _index = list(canon_tb.column('docid')).index(_id)
    probabilities[_index] = probability

In [ ]:
# Add this probability as a new column to our table of canonic texts

canon_tb['P(reviewed)'] = probabilities

In [ ]:
# Visualize

canon_tb.scatter('firstpub','P(reviewed)', fit_line=True)

In [ ]:
##  Q. Two of the prestigious texts are assigned less than 50% probability
##     that they were reviewed. How do we make sense of that?